# Entrenamiento de modelos, evaluacion y seleccion.

**Elaborado por:** Jacqueline Fernández Ramírez, Jazmín Fernández Ramírez y Nina Odoux.

Máster universitario de Ciencia de Datos (MUCD) - CUNEF Universidad.


### La Importancia de Entrenar, Evaluar y Seleccionar

Desarrollar un modelo efectivo de machine learning requiere entrenar, evaluar y seleccionar cuidadosamente el modelo con mejor rendimiento. El entrenamiento permite que el modelo aprenda patrones a partir de los datos, la evaluación asegura que generalice bien a datos nuevos, y la selección identifica la solución más confiable y robusta. Juntos, estos pasos garantizan que el modelo final ofrezca resultados precisos y consistentes, abordando el problema de manera efectiva y maximizando su aplicabilidad en el mundo rea
l
* Entrenar: Es donde el modelo aprende patrones de los datos. Esto le permite hacer predicciones precisas y generalizar a nuevos datos.

* Evaluar: Comprobamos si el modelo funciona bien con datos nuevos usando métricas adaptadas. Esto evita que el modelo sea poco confiable.

* Seleccionar: Comparamos varios modelos para elegir el más efectivo y balanceado para el problema.

Siguiendo estos pasos, garantizamos un modelo que sea útil y fiable en el mundo real.



In [27]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
seed = np.random.seed(123)
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics

import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from boruta import BorutaPy 

## ejemplo tipo de LASSO y de entrenamiento al feature engineering con subjconjunto del dataset 

In [32]:
#EJEMPLO DE LASSO Y COEFICIENTES CON SUBCONJUNTO DE DATA PARA ENTRENARSE AL FEATURE ENGINEERING Y SLECCIONAR VARIABLES 

# Cargar el dataset
X_train = pd.read_csv("../data/preprocessed_data/X_train_final.csv")
X_test = pd.read_csv("../data/preprocessed_data/X_test_final.csv")
y_train =pd.read_csv("../data/preprocessed_data/y_train.csv")
y_test =pd.read_csv("../data/preprocessed_data/y_test.csv")


# Tomar un subconjunto aleatorio de 1000 observaciones
X_train_subset = X_train.sample(n=1000, random_state=seed)
X_test_subset = X_test.sample(n=1000, random_state=seed)
y_train_subset =y_train.sample(n=1000, random_state=seed)
y_test_subset =y_test.sample(n=1000, random_state=seed)


# Mostrar el subconjunto
print(X_train_subset)
print(X_test_subset)
print(y_train_subset)
print(y_test_subset)


# Entrenar el modelo Lasso con alpha=0.001
lasso_orig = Lasso(alpha=0.001)
lasso_orig.fit(X_train_subset, y_train_subset)

# Obtener los coeficientes del modelo Lasso
lasso_coefs = lasso_orig.coef_  # Coeficientes entrenados con alpha=0.001



# NO TODO LO QUE ES EXPLICABLE ES INTERPRETABLE 

#MODELOS DE REGRESSION LINEAL SENCILLOS SON EXPLICABLES E INTERPRETABLES 

#LA INTERPRETABILIDAD ES COMPRENDER EL FUNCIONAMIENTO DEL MODELO INTERNO 
#UNA RED N ES INTERPRETABLE PERO NO EXPLICABLE 



        flag_document_8  apartments_avg  live_city_not_work_city  \
68036         -0.245406       -0.986425                -0.997392   
168756        -0.245406        1.013259                 1.002211   
234695        -0.245406       -0.986425                -0.997392   
85636         -0.245406       -0.986425                -0.997392   
1888          -0.245406       -0.986425                -0.997392   
...                 ...             ...                      ...   
168132        -0.245406       -0.986425                -0.997392   
6721          -0.245406        1.014361                 1.003460   
186286        -0.245406        1.012738                 1.002043   
61171         -0.245406       -0.986425                -0.997392   
81178         -0.245406       -0.986425                -0.997392   

        nonlivingapartments_medi  fondkapremont_mode_reg oper spec account  \
68036                  -0.547233                                 -0.828364   
168756                 -0.5

In [ ]:

#pipeline de numericos
#pipeline de categoricas 


#preprocessor column transformer 


















# Define models to test
models = {
    "Logistic Regression (GLM)": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "LightGBM": LGBMClassifier(),
    "SVM": SVC(probability=True)  # Enable probability for metrics like ROC-AUC
}

# Initialize a results dictionary
results = {}

# Evaluate each model
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize the data
        ('classifier', model)
    ])

    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')  # Use F1-score as example
    print(f"{name} - CV F1 Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

    # Fit the pipeline on training data and evaluate on the test set
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Collect performance metrics
    test_metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred)
    }
    results[name] = test_metrics

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(results_df)

# Visualize results
import matplotlib.pyplot as plt
results_df[['Accuracy', 'Precision', 'Recall', 'F1 Score']].plot(kind='bar', figsize=(10, 6))
plt.title('Model Performance Metrics')
plt.ylabel('Score')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.show()